In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size=3, input_shape=(32,32,3), activation='relu', padding='same'))
model.add(MaxPooling2D(2))
model.add(Conv2D(32, kernel_size=3, activation='relu', padding='same'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(64, activation = "relu"))        
model.add(Dense(32, activation = "relu"))
model.add(Dense(16, activation = "relu")) 
model.add(Dense(1, activation='relu'))

model.compile(optimizer=Adam(lr=0.0001), loss='mse')

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
y_train

In [ ]:
images = [file for file in os.listdir('../image_test/raw')]

In [ ]:
os.mkdir('../image_test/new')

for idx, img in enumerate(images):
    if idx != len(images)-1:
        cur_img = cv2.imread('../image_test/raw/' + images[idx])
        next_img = cv2.imread('../image_test/raw/' + images[idx+1])
        of_img = opticalFlowDense(cur_img, next_img)
        cv2.imwrite('../image_test/new/' + images[idx],
                    of_img)
    
    else:
        cur_img = cv2.imread('../image_test/raw/' + images[idx])
        of_img = opticalFlowDense(cur_img, cur_img)
        cv2.imwrite('../image_test/new/' + images[idx],
                    of_img)

In [ ]:
def opticalFlowDense(image_current, image_next):
    
    gray_current = cv2.cvtColor(image_current, cv2.COLOR_RGB2GRAY)
    gray_next = cv2.cvtColor(image_next, cv2.COLOR_RGB2GRAY)
    
    
    hsv = np.zeros((366, 366, 3))
    # set saturation
    hsv[:,:,1] = cv2.cvtColor(image_next, cv2.COLOR_RGB2HSV)[:,:,1]
 
    # Flow Parameters
#     flow_mat = cv2.CV_32FC2
    flow_mat = None
    image_scale = 0.5
    nb_images = 1
    win_size = 15
    nb_iterations = 2
    deg_expansion = 5
    STD = 1.3
    extra = 0

    # obtain dense optical flow paramters
    flow = cv2.calcOpticalFlowFarneback(gray_current, gray_next,  
                                        flow_mat, 
                                        image_scale, 
                                        nb_images, 
                                        win_size, 
                                        nb_iterations, 
                                        deg_expansion, 
                                        STD, 
                                        0)
                                        
        
    # convert from cartesian to polar
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])  
        
    # hue corresponds to direction
    hsv[:,:,0] = ang * (180/ np.pi / 2)
    
    # value corresponds to magnitude
    hsv[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # convert HSV to float32's
    hsv = np.asarray(hsv, dtype= np.float32)
    rgb_flow = cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)

    
    return rgb_flow

In [ ]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it

old_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

In [ ]:
p0[1]

In [ ]:
xs = [point[0][0] for point in p0]
ys = [point[0][1] for point in p0]

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

plt.imshow(image)
plt.scatter(xs, ys, cmap='viridis')

In [ ]:
image1 = plt.imread('../image_data/train/ojw_020.jpg')
#image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

image2 = plt.imread('../image_data/train/ojw_021.jpg')
#image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

In [ ]:
cap = cv.VideoCapture("storm_example.avi")
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    cv.imshow('frame2',bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png',frame2)
        cv.imwrite('opticalhsv.png',bgr)
    prvs = next

In [ ]:
import cv2 as cv